# Testing neural networks approach to numerical integration on Genz numerical integration testing package 

These tests are dedicated to research of dependency of k size of neural network (size of single hidden layer).

The tests will be done on 2D functions

## Library import

In [ ]:
%run ../skuld/skuld.py      # skuld NNI library
%run ../utils/plots.py      # plotting functions
%run ../utils/integrate.py  # 'classic' numerical integration function

import pickle

# Part 1: k-size tests

## 2D testing function

In [ ]:
def test_2d_integration(h, ej, func, func_float, a, b, func_name, logplot=False):
    global input_size, hidden_size, u, c
  
    X_init, y_init = generate_data(func, a, b, 200, input_size, u, c)
    if logplot:
        plot_2d_function_heatmap_with_log(X_init, y_init, func_name)
    else:
        plot_2d_function_heatmap(X_init, y_init, func_name)
    X, y = scale_data(X_init, y_init, n_dim=2)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)  
    model = MLP(input_size, hidden_size)
    model.compile(criterion=nn.MSELoss(), optimizer=optim.Adam(model.parameters(), lr=learning_rate))
    train_history = model.train(x_train, y_train, num_epochs, verbose=False)
    test_loss = model.test(x_test, y_test)
    print(f"Test Loss: {test_loss:.10f}")
    
    nni_scaled = NeuralNumericalIntegration.integrate(model, a, b, n_dims=2)
    nni_result = descale_result(float(nni_scaled[0]), X_init, y_init, frange=(0, 1), n_dim=2)
    result_quad = integrate_2d_nquad(func_float, a, b, u, c)
    result_trapz = integrate_2d_trapz(func, a, b, 100, u, c)
   
    return nni_result, result_quad, result_trapz

## Data operations functions

In [ ]:
def save_data(data, filename="data.pkl"):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

def load_data(filename="data.pkl"):
    with open(filename, 'rb') as f:
        loaded_data = pickle.load(f)
    print(f"Data loaded from {filename}")
    
    return loaded_data

def calculate_mae(results):
    absolute_errors = [abs(a - b) for a, b, _ in results]
    mae = sum(absolute_errors) / len(absolute_errors)
    
    return mae

## Hyperparams for 2D

In [ ]:
input_size = 2
initial_hidden_size = 5
hidden_size = initial_hidden_size         # initialy k = 10
learning_rate = 0.001
num_epochs = 5000

tests_num = 20

a_ = [0.0, 0.0]
b_ = [1.0, 1.0]

all_results = []

## 1. Oscillatory 2D

In [ ]:
func_name = 'Oscillatory 2D'
h = 100
ej = 1
results = []

u = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c = [fraction * value for value in cs]

def osc_2d(X, u, c):
    sum_ = 0
    for i in range(input_size):
        sum_ += c[i] * X[:, i]

    return torch.cos(2 * math.pi * u[0] + sum_)
    
def osc_2d_float(X, u, c):
    sum_ = 0
    for i in range(input_size):
        sum_ += c[i] * X[i]
    return math.cos(2 * math.pi * u[0] + sum_)

for i in range(tests_num):
    hidden_size = initial_hidden_size * (i + 1) # k is updated each iteration k_new = k_old + 10
    print(f"\n\033[1mIteration {i+1} is running! k = {hidden_size}\033[0m")
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))

print(h, ej, u, c)

plot_test_results(results)

all_results.append(results)

## 2. Product Peak 2D

In [ ]:
func_name = 'Product Peak 2D'
h = 150
ej = 1
results = []

u = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c = [fraction * value for value in cs]

def prod_peek_2d(X, u, c):
    prod_ = 1
    for i in range(2):
        prod_ *= (c[i] ** (-2) + (X[:, i] - u[i])**2) ** (-1)
    return prod_

def prod_peek_2d_float(X, u, c):
    prod_ = 1
    for i in range(2):
        prod_ *= (c[i] ** (-2) + (X[i] - u[i])**2) ** (-1)
    return prod_

for i in range(tests_num):
    hidden_size = initial_hidden_size * (i + 1) # k is updated each iteration k_new = k_old + 10
    print(f"\n\033[1mIteration {i+1} is running! k = {hidden_size}\033[0m")
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))

print(h, ej, u, c)

plot_test_results(results)

all_results.append(results)

## 3. Corner Peak 2D

In [ ]:
func_name = 'Corner Peak 2D'
h = 600
ej = 1
results = []

u = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c = [fraction * value for value in cs]

def corn_peek_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] * X[:, i]
    return (1 + sum_) ** (-3)

def corn_peek_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] * X[i]
    return (1 + sum_) ** (-3)

for i in range(tests_num):
    hidden_size = initial_hidden_size * (i + 1) # k is updated each iteration k_new = k_old + 10
    print(f"\n\033[1mIteration {i+1} is running! k = {hidden_size}\033[0m")
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))

print(h, ej, u, c)

plot_test_results(results)

all_results.append(results)

## 4. Gaussian 2D

In [ ]:
func_name = 'Gaussian 2D'
h = 150
ej = 1
results = []

u = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c = [fraction * value for value in cs] 

def gauss_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += (c[i] ** 2) * ((X[:, i] - u[i]) ** 2)
    return torch.exp(-sum_)

def gauss_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += (c[i] ** 2) * ((X[i] - u[i]) ** 2)
    return np.exp(-sum_)

for i in range(tests_num):
    hidden_size = initial_hidden_size * (i + 1) # k is updated each iteration k_new = k_old + 10
    print(f"\n\033[1mIteration {i+1} is running! k = {hidden_size}\033[0m")
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))

print(h, ej, u, c)

plot_test_results(results)

all_results.append(results)

## 5. Continuous 2D

In [ ]:
func_name = 'Continuous 2D'
h = 100
ej = 1
results = []

u = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c = [fraction * value for value in cs] 

def cont_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i]  * abs(X[:, i] - u[i])
    return torch.exp(-sum_)

def cont_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] ** 2 * abs(X[i] - u[i]) ** 2
    return np.exp(-sum_)

for i in range(tests_num):
    hidden_size = initial_hidden_size * (i + 1) # k is updated each iteration k_new = k_old + 10
    print(f"\n\033[1mIteration {i+1} is running! k = {hidden_size}\033[0m")
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))

print(h, ej, u, c)

plot_test_results(results)

all_results.append(results)

## 6. Discontinuous 2D

In [ ]:
func_name = 'Discontinuous 2D'
h = 16
ej = 1
results = []

u = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c = [fraction * value for value in cs]

def disco_2d(X, u, c):
    result = torch.zeros(X.shape[0])
    for i in range(X.shape[0]):
        x = X[i, 0].item()
        
        if x > u[0]:
            result[i] = 0.0
        else:
            result[i] = math.exp(c[0] * x)

    return result

def disco_1d_float(X, u, c):
    x1, x2 = X

    if x1 > u[0] or x2 > u[1]:
        return 0.0
    else:
        return math.exp(c[0] * x1 + c[1] * x2)

for i in range(tests_num):
    hidden_size = initial_hidden_size * (i + 1) # k is updated each iteration k_new = k_old + 10
    print(f"\n\033[1mIteration {i+1} is running! k = {hidden_size}\033[0m")
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))

print(h, ej, u, c)

plot_test_results(results)

all_results.append(results)

# Calculation of MAEs for the research

In [ ]:
all_maes = []

for results in all_results:
    results_list = []
    chunk_size = 3
    for i in range(0, len(results), chunk_size):
        results_list.append(results[i:i + chunk_size])
    maes = [calculate_mae(results) for results in results_list]
    all_maes.append(maes)

In [ ]:
for maes in all_maes:
    for i, mae in enumerate(maes, start=1):
        print(f"{mae:.8f}")
    print('\n')

In [ ]:
save_data(all_results, filename="cucumbers/all-rests.pkl")
loaded_data = load_data(filename="cucumbers/all-rests.pkl")

loaded_data == all_results

In [ ]:
save_data(all_maes, filename="cucumbers/all-maes.pkl")
loaded_data = load_data(filename="cucumbers/all-maes.pkl")

loaded_data == all_maes